In [55]:
import json
import argparse
import pandas as pd
from tqdm import tqdm

In [4]:
csv_path = '/home/nakai/project/aigc/llm/dataset/聊天内容/40v40聊天记录.csv'

In [74]:
!wc -l /home/nakai/project/aigc/llm/dataset/聊天内容/40v40聊天记录.csv

10361325 /home/nakai/project/aigc/llm/dataset/聊天内容/40v40聊天记录.csv


In [53]:
df = pd.read_csv(csv_path, nrows=3000000)

/tmp/ipykernel_588342/1409326339.py:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, nrows=3000000)


In [54]:
conversations = {}
# 将消息组合成长文本
for _, row in tqdm(df.iterrows()):
    from_uid = row['src_id']
    to_uid = row['dest_id']
    msg = str(row['content']).strip()
    msg = str(from_uid) + '###' + str(row['content']).strip()

    # 将from_uid和to_uid按照字母顺序组合成一个元组，确保对话参与者的组合是唯一的
    participants = tuple(sorted([from_uid, to_uid]))

    if participants not in conversations:
        conversations[participants] = []
    if msg.find("{") >= 0:
        continue
    if len(conversations[participants]) > 0:
        last_msg = conversations[participants][-1]
        if last_msg.split('###')[0] == msg.split('###')[0]:
            conversations[participants][-1] = last_msg +","+msg.split('###')[-1]
        else:
            conversations[participants].append(msg)
    else:     
        conversations[participants].append(msg)

3000000it [01:52, 26778.20it/s]


In [68]:
# 构造单轮会话数据集，无提示词
'''
 {
    "instruction": "你还能做什么?",
    "input": "",
    "output": "我可以替你写代码，作文等",
    "history": []
  },
'''
cnt = 0
with open('/home/nakai/project/aigc/llm/LLaMA-Factory/data/chat_dataset/chat.json', mode='w') as tf:
    tf.write('[\n')
    corpus = []
    for item in conversations.values():
        if len(item) <= 1:
            continue
        for idx in range(1, len(item)):
            cnt += 1
            sep = ''
            if cnt > 1:
                sep = ','
            ip = item[idx-1].split('###')[-1].replace('\n', '').replace('"', '').replace('/', '').replace('\\', '').replace('^M', '')
            ot = item[idx].split('###')[-1].replace('\n', '').replace('"', '').replace('/', '').replace('\\', '').replace('^M', '')
            tf.write(f"""{sep}{{
    "instruction": "",
    "input": "{ip}",
    "output": "{ot}",
    "history": []
  }}
""")
    tf.write(']')
    